#<center>Web Server Logs Analysis</center>

# Team Details
## Group: D

---

## Team Members:
<ul>
  
<li>Khadar wali Afghan</li>
<li>Darshan Patgar</li>
<li>Kiran Yaligar</li>
<li>Sharath S Hiremath</li>
<li>Vandana Hegde</li>
</ul>

<h3>Problem Statement:</h3><t><p>In this project, we use Hadoop, Spark and SparkSql to process the Web Server logs dataset to get more insights on the log data. 
  </p><p>As part of this, we create Azure Data Factory and connect our storage with databrick to perform analysis on the data and get more insights of the data and also Build Scala code, submit Spark jobs and SparkSql Queries using the dataset.
</p>

<h3>Objectives:</h3>
</br>
<ul>
<li>Create an Azure Data Factory.</li>
<li>Create a Storage Resource and store the dataset.</li>
<li>Connect Databrick Activity to the Pipline in Azure Data Factory.</li>
<li>Clean the Dataset and save as CSV file.</li>
<li>Read the CSV file as Resilient Distributed Dataset(RDD) format.</li>
<li>Perform Spark jobs.</li>
<li>Perform SparkSql Qeries</li>
<li>Analyze the data using PowerBi</li>

<h2>Dataset Description</h2>
<br>
<ul>
  <li>Web sever logs contain information on any event that was registered/logged.</li>
  <li>This contains a lot of insights on website visitors, behavior, crawlers accessing the site, business insights, security issues, and more</li>
  <li>3.3GB of logs from an Iranian ecommerce website zanbil.ir.
</li>
  <li>Number of records: 10,48,576
</li>
  <li>The data is present in Text file.
</li>
  <li>This data contain information such as host IP address, datetime, request, reply code ,bytes , user agent
</li>
  <li>Example:<br><p> 54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"
</li>

<h2>Data Pre-Processing(Cleaning the data)</h2></br>
<li>This data contain information such as host IP address, datetime, request, reply code ,bytes , user agent.,Etc</li>
<li>The first feild is the host ip address from were the request for the resource is generated.</li>
<li>The Second feild is the date and time data on which date and time the request was sent. this feild is in DAY:MONTH:YEAR:HH:MM:SS </li>
<li>The Third feild is haveing multiple information such as method(GET, POST, PUT), the request of the file or content in encrypted in url format, and the protocol used by the browser to send the request.</li>
<li>The next feild contain the response code which specify the the status of the request.</li>
<li>Next feild is the byte size of the resource requested by the user agent</li>
<li>The last feild is the user agent, with the help of this data we can determine weather the request was generated by a mobile, windows or a Bot, also it has the the comapatablity of the broswer like Mozilla/5.0 and in some case we also get the entire data of the mobile phone.

weblogstext = sc.read.text(DataPath)
<br></br>
<li>First we read the file using sparkContext as it is a textfile we will read it as text file

weblogstext.take(1)
</br></br>
<li>output>>
</br>
[Row(value='54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"')]

blanks = rdd1.map(lambda x:','.join(x or '0' for x in x.split(' ')))
</br></br>
<li> Removing blank records if there are any, We didnt had an blank records.

In [0]:
# Split the row into list
import csv
from io import StringIO
def split(line):
    reader = csv.reader(StringIO(line))
    return next(reader)

weblogsrdd=blanks.map(split)
weblogsrdd.take(10)

<li>spliting each records to get all the feilds seprated by ',' required for the analysis
  </li></br></br>
  <b>Output>></b>
  ['31.56.96.51',
  '-',
  '-',
  '[22/Jan/2019:03:56:16',
  '+0330]',
  'GET,/image/60844/productModel/200x200,HTTP/1.1',
  '200',
  '5667',
  'https://www.zanbil.ir/m/filter/b113',
  'Mozilla/5.0,(Linux;,Android,6.0;,ALE-L21,Build/HuaweiALE-L21),AppleWebKit/537.36,(KHTML,,like,Gecko),Chrome/66.0.3359.158,Mobile,Safari/537.36',
  '-'],

<li>After asigning names to each feild with the help of Row function from pyspark.sql</li></br></br>
<b>Output >></b>
[Row(client='54.36.149.41', datetime='22/Jan/2019:03:56:14', method=['GET', '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', '27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', 'p53', 'HTTP/1.1'], rescode='200', size='30577', user_agent='Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)')]

<li> In the above output we still need to take data from method feild as it conatin sub information like protocol,requested resource and the method.</li>

In [0]:
# Taking the records which contain all the 3 required feilds i.e method, requested resource, protocol used.
df12 = dataframe.filter(lambda x: len(x.method) >= 3)

<b>Output>></b>

[Row(client='54.36.149.41', datetime='22/Jan/2019:03:56:14', rescode='200', method='GET', request='/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', protocol='HTTP/1.1', size='30577', user_agent='Mozilla/5.0,(compatible;,AhrefsBot/6.1;,+http://ahrefs.com/robot/)')]

<li> Still the user agent needs to be processed, with the help of this data we will determine weather the user is mobile, windows, Bot,.etc.</li>

In [0]:
dataframe2 = dataframe1.map(lambda x:Row(host=x[0],datetime=x[1],rescode=x[2],method=x[3],request=x[4],protocol=x[5],size=x[6],user_agent = 'Bot' if 'AhrefsBot' in x[7] else 'Mobile' if 'Mobile' in x[7] else "Windows" if 'Windows' in x[7] else "Bot" if "bingbot" in x[7] else "Bot" if 'Googlebot' in x[7] else "Applebot" if 'Applebot' in x[7] else 'Unknown' ) )

<b>Output>></b>
[Row(host='54.36.149.41', datetime='22/Jan/2019:03:56:14', rescode='200', method='GET', request='/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', protocol='HTTP/1.1', size='30577', user_agent='Bot')]

In [0]:
#Save the dataframe in csv format with header
WebLogsFile.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/content/drive/MyDrive/WebLogData/mywebdata.csv")

In [0]:
storageaccount = "weblogstorage" #Name of the storage account where the data is stored.
storagekey = "cRvy8hMUD5azpX9GvW8MBztK6LHsIJCA1wNKlyx9G86NdbMlgYojgkNh9f6ojr8P/zTAQUPqYrGI+AStuAsIWQ==" #Storage Key
containername = "webcontainer" #Name of the container 

In [0]:
#set the spark configuration
spark.conf.set(f"fs.azure.account.key.{storageaccount}.blob.core.windows.net",storagekey)

In [0]:
#Directories and files present in the container
dbutils.fs.ls(f"wasbs://{containername}@{storageaccount}.blob.core.windows.net/")

Out[3]: [FileInfo(path='wasbs://webcontainer@weblogstorage.blob.core.windows.net/input/', name='input/', size=0, modificationTime=0),
 FileInfo(path='wasbs://webcontainer@weblogstorage.blob.core.windows.net/output/', name='output/', size=0, modificationTime=0),
 FileInfo(path='wasbs://webcontainer@weblogstorage.blob.core.windows.net/weblogs.txt', name='weblogs.txt', size=1364423315, modificationTime=1661163498000)]

In [0]:
#creating dataframe by reading csv file in the container of Azure blob storage
df = spark.read.option("header","true").option("inferSchema","true").option("delimiter",",").csv(f"wasbs://{containername}@{storageaccount}.blob.core.windows.net/output/WebLogData.csv")

In [0]:
#converting dataframe into RDD
rdd1=df.rdd
rdd1.take(1)

Out[7]: [Row(host='54.36.149.41', datetime='22/Jan/2019:03:56:14', rescode=200, method='GET', request='/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', protocol='HTTP/1.1', size=30577, user_agent='Bot')]

<h3>Web Server Log Analysis using Spark Jobs</h3>

<b>0)Get the successful requests made by host

In [0]:
rdd1.filter(lambda x:x.rescode == 200).take(10)

Out[8]: [Row(host='54.36.149.41', datetime='22/Jan/2019:03:56:14', rescode=200, method='GET', request='/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84', protocol='HTTP/1.1', size=30577, user_agent='Bot'),
 Row(host='31.56.96.51', datetime='22/Jan/2019:03:56:16', rescode=200, method='GET', request='/image/60844/productModel/200x200', protocol='HTTP/1.1', size=5667, user_agent='Mobile'),
 Row(host='31.56.96.51', datetime='22/Jan/2019:03:56:16', rescode=200, method='GET', request='/image/61474/productModel/200x200', protocol='HTTP/1.1', size=5379, user_agent='Mobile'),
 Row(host='40.77.167.129', datetime='22/Jan/2019:03:56:17', rescode=200, method='GET', request='/image/14925/productModel/100x100', protocol='HTTP/1.1', size=1696, user_agent='Bot'),
 Row(host='91.99.72.15', datetime='22/Jan/2019:03:56:17', rescode=200, method='GET', request='/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%D8%B1-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C-%D9%BE%D8%B1%D9%86%D8%B3%D9%84%DB%8C-%D9%85%D8%AF%D9%84-PR257AT', protocol='HTTP/1.1', size=41483, user_agent='Windows'),
 Row(host='40.77.167.129', datetime='22/Jan/2019:03:56:17', rescode=200, method='GET', request='/image/23488/productModel/150x150', protocol='HTTP/1.1', size=2654, user_agent='Bot'),
 Row(host='40.77.167.129', datetime='22/Jan/2019:03:56:18', rescode=200, method='GET', request='/image/45437/productModel/150x150', protocol='HTTP/1.1', size=3688, user_agent='Bot'),
 Row(host='40.77.167.129', datetime='22/Jan/2019:03:56:18', rescode=200, method='GET', request='/image/576/article/100x100', protocol='HTTP/1.1', size=14776, user_agent='Bot'),
 Row(host='66.249.66.194', datetime='22/Jan/2019:03:56:18', rescode=200, method='GET', request='/filter/b41', protocol='HTTP/1.1', size=34277, user_agent='Bot'),
 Row(host='40.77.167.129', datetime='22/Jan/2019:03:56:18', rescode=200, method='GET', request='/image/57710/productModel/100x100', protocol='HTTP/1.1', size=1695, user_agent='Bot')]

<b>1)Get the count of request made by each host

In [0]:
rdd1.map(lambda x:x[0]).countByValue()

Out[9]: defaultdict(int,
 {'54.36.149.41': 157,
 '31.56.96.51': 78,
 '40.77.167.129': 427,
 '91.99.72.15': 38694,
 '66.249.66.194': 353483,
 '207.46.13.136': 926,
 '178.253.33.51': 7,
 '66.249.66.91': 314522,
 '5.78.198.52': 50,
 '34.247.132.53': 6830,
 '54.36.149.70': 191,
 '2.177.12.140': 18,
 '89.199.193.251': 1,
 '66.111.54.249': 135,
 '207.46.13.115': 1785,
 '173.249.54.67': 25,
 '5.112.52.254': 5,
 '5.211.97.39': 474,
 '2.179.141.98': 6,
 '54.36.148.87': 134,
 '17.58.102.43': 21698,
 '51.15.15.54': 160,
 '54.36.149.17': 140,
 '5.62.206.249': 3,
 '54.36.148.18': 174,
 '54.36.148.17': 174,
 '2.185.221.79': 2,
 '207.46.13.104': 86,
 '54.36.148.32': 134,
 '54.36.149.35': 138,
 '172.20.2.174': 11303,
 '5.160.157.20': 13526,
 '54.36.148.161': 138,
 '157.55.39.245': 780,
 '5.209.200.218': 35,
 '54.36.149.63': 154,
 '54.36.148.10': 144,
 '204.18.198.248': 47,
 '66.249.64.66': 90,
 '5.78.180.75': 29,
 '54.36.148.117': 145,
 '104.194.24.33': 14,
 '54.36.148.232': 164,
 '46.224.77.32': 12,
 '54.36.149.92': 181,
 '54.36.149.58': 189,
 '54.36.149.83': 127,
 '66.249.66.197': 5899,
 '54.36.149.84': 159,
 '216.244.66.248': 544,
 '54.36.149.105': 166,
 '54.36.149.16': 184,
 '54.36.149.7': 190,
 '54.36.149.77': 143,
 '54.36.149.102': 168,
 '5.127.80.240': 1,
 '5.123.89.112': 8,
 '130.185.74.243': 5380,
 '54.36.148.14': 156,
 '54.36.148.197': 165,
 '54.36.149.54': 145,
 '54.36.148.70': 141,
 '5.217.191.165': 4,
 '157.55.39.175': 48,
 '178.131.133.79': 1,
 '5.209.127.187': 12,
 '54.36.148.135': 175,
 '5.208.84.157': 1,
 '5.123.209.223': 6,
 '54.36.148.57': 151,
 '54.36.148.28': 172,
 '54.36.148.185': 161,
 '54.36.148.176': 176,
 '54.36.148.72': 155,
 '66.249.83.1': 490,
 '54.36.149.86': 127,
 '89.47.79.75': 135,
 '66.249.83.5': 478,
 '66.249.83.91': 2186,
 '66.249.83.93': 2118,
 '66.249.83.92': 2240,
 '54.36.148.143': 166,
 '66.249.83.7': 407,
 '54.36.148.118': 158,
 '54.36.148.65': 183,
 '54.36.148.22': 156,
 '46.51.119.113': 1,
 '54.36.149.68': 155,
 '54.36.148.23': 139,
 '54.36.148.211': 173,
 '54.36.149.11': 137,
 '54.36.148.137': 151,
 '5.115.150.169': 1,
 '5.123.232.17': 2,
 '17.58.97.4': 1,
 '54.36.148.237': 181,
 '52.7.238.253': 1019,
 '91.251.165.118': 36,
 '66.249.66.195': 4722,
 '5.115.54.117': 138,
 '95.81.118.56': 82,
 '54.36.149.59': 147,
 '54.36.148.100': 161,
 '40.77.193.190': 3,
 '188.25.76.139': 1,
 '40.77.193.47': 8,
 '63.143.42.246': 4500,
 '40.77.195.42': 4,
 '40.77.253.31': 30,
 '37.202.154.8': 3,
 '54.36.148.29': 146,
 '192.119.14.199': 369,
 '54.36.149.6': 161,
 '5.127.227.69': 1,
 '54.36.148.88': 126,
 '5.112.80.163': 1,
 '89.198.116.98': 2,
 '93.113.124.199': 1,
 '46.229.161.131': 229,
 '54.36.148.125': 145,
 '5.126.60.10': 255,
 '54.36.148.113': 168,
 '89.198.193.18': 7,
 '151.235.137.158': 4,
 '5.116.222.247': 1,
 '54.36.148.159': 164,
 '35.182.176.64': 1,
 '69.162.124.229': 2256,
 '63.143.42.243': 1107,
 '54.36.148.248': 166,
 '64.233.172.175': 1070,
 '5.127.189.90': 4,
 '63.143.42.242': 2311,
 '69.162.124.234': 2206,
 '54.36.148.217': 157,
 '54.36.149.106': 163,
 '5.123.174.57': 40,
 '54.36.149.2': 185,
 '54.36.149.97': 147,
 '54.36.148.114': 146,
 '192.15.21.99': 1,
 '83.123.86.60': 36,
 '54.36.148.254': 160,
 '5.123.213.242': 2,
 '54.36.148.55': 138,
 '54.36.149.73': 186,
 '54.36.148.180': 139,
 '54.36.149.4': 143,
 '5.114.139.12': 1,
 '2.181.245.181': 1,
 '5.126.170.223': 1,
 '104.36.19.231': 11,
 '54.36.148.230': 182,
 '54.36.148.224': 198,
 '54.36.149.10': 156,
 '109.125.150.157': 6,
 '54.36.148.21': 161,
 '54.36.148.90': 176,
 '54.36.149.14': 151,
 '13.66.139.0': 1177,
 '54.36.148.149': 159,
 '54.36.148.11': 155,
 '54.36.149.49': 178,
 '54.36.148.154': 156,
 '54.36.148.200': 157,
 '54.36.148.173': 165,
 '89.198.184.125': 12,
 '104.248.18.164': 2,
 '5.123.139.174': 4,
 '54.36.148.48': 173,
 '46.4.203.149': 4759,
 '74.6.168.162': 10,
 '51.38.125.215': 83,
 '74.6.168.254': 7,
 '54.36.148.171': 172,
 '89.32.98.244': 18,
 '83.120.122.254': 19,
 '5.115.82.113': 1,
 '54.36.149.38': 153,
 '54.36.148.164': 161,
 '

<b>2)Which host(ip address) has made maximum number of request

In [0]:
a=rdd1.map(lambda x:x[0]).countByValue()
b=max(a,key=a.get)
print(b)

66.249.66.194

<b>3)Get maximum agent used by host to make the request

In [0]:
c=rdd1.map(lambda x:x[7]).countByValue()
d=max(c,key=c.get)
print(d)

Windows

<b>4)Get count of the logs user agent wise

In [0]:
rdd1.map(lambda x:x[7]).countByValue()

Out[12]: defaultdict(int,
 {'Bot': 606189,
 'Mobile': 3992853,
 'Windows': 5137742,
 'Unknown': 606529,
 'Applebot': 21783})

<b>5)Get count the number of successful repsonses

In [0]:
rdd1.filter(lambda x:x[2]==200).count()

Out[13]: 9579824

<b>6)In how many logs the server can not find the requested resource

In [0]:
rdd1.filter(lambda x:x[2]==404).count()

Out[14]: 105011

<b>7)top viewed pages or content

In [0]:
e=rdd1.map(lambda x:x[4]).countByValue()
f=max(e,key=e.get)
print(f)

/settings/logo

<b>8)Get the number of distinct users

In [0]:
x=rdd1.map(lambda x:x[0]).distinct()
x.count()

Out[16]: 258594

<h3>Web Server Log Analysis using Spark SQL</h3>

In [0]:
#importing modules
from pyspark.sql import SQLContext, Row

In [0]:
#create SQL context in spark
sqlC=SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(

In [0]:
#Check SQL Context
sqlC

Out[19]: <pyspark.sql.context.SQLContext at 0x7f3d9002cd00>

In [0]:
#Register table name as "WebLogs"
df.registerTempTable("WebLogs")

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

In [0]:
#spark sql query to get the number of request made by each user in descending order
sqlC.sql("select host, Count(host) as c from WebLogs group by host order by c desc").collect()

Out[21]: [Row(host='66.249.66.194', c=353483),
 Row(host='66.249.66.91', c=314522),
 Row(host='151.239.241.163', c=92475),
 Row(host='66.249.66.92', c=88332),
 Row(host='91.99.30.32', c=45979),
 Row(host='104.222.32.91', c=42058),
 Row(host='91.99.72.15', c=38694),
 Row(host='91.99.47.57', c=38612),
 Row(host='5.78.190.233', c=37204),
 Row(host='195.181.168.181', c=27979),
 Row(host='23.101.169.3', c=27800),
 Row(host='195.181.168.164', c=24044),
 Row(host='66.249.66.93', c=22146),
 Row(host='17.58.102.43', c=21698),
 Row(host='104.222.32.94', c=16706),
 Row(host='5.160.157.20', c=13526),
 Row(host='5.117.116.238', c=13427),
 Row(host='5.160.221.38', c=12058),
 Row(host='172.20.2.174', c=11303),
 Row(host='40.77.167.156', c=10231),
 Row(host='207.46.13.60', c=9958),
 Row(host='40.77.167.205', c=9860),
 Row(host='130.185.76.185', c=9534),
 Row(host='217.219.76.102', c=8271),
 Row(host='207.46.13.49', c=6970),
 Row(host='5.116.203.8', c=6948),
 Row(host='34.247.132.53', c=6830),
 Row(host='157.55.39.146', c=6030),
 Row(host='66.249.66.94', c=5985),
 Row(host='66.249.66.197', c=5899),
 Row(host='109.201.26.239', c=5684),
 Row(host='130.185.74.243', c=5380),
 Row(host='207.46.13.132', c=5255),
 Row(host='194.225.55.136', c=5039),
 Row(host='46.4.203.149', c=4759),
 Row(host='66.249.66.195', c=4722),
 Row(host='40.77.167.170', c=4648),
 Row(host='207.46.13.137', c=4612),
 Row(host='91.99.72.148', c=4577),
 Row(host='212.80.0.3', c=4562),
 Row(host='63.143.42.246', c=4500),
 Row(host='195.181.169.141', c=4373),
 Row(host='40.77.167.13', c=4332),
 Row(host='40.77.167.127', c=4147),
 Row(host='162.223.91.56', c=3877),
 Row(host='217.218.130.15', c=3857),
 Row(host='78.109.197.68', c=3821),
 Row(host='185.186.240.178', c=3802),
 Row(host='185.155.236.247', c=3798),
 Row(host='2.178.8.199', c=3713),
 Row(host='104.222.32.84', c=3681),
 Row(host='40.77.167.191', c=3678),
 Row(host='91.106.70.17', c=3671),
 Row(host='46.224.113.43', c=3621),
 Row(host='178.252.132.98', c=3592),
 Row(host='185.11.88.198', c=3588),
 Row(host='116.203.60.232', c=3471),
 Row(host='77.104.75.68', c=3416),
 Row(host='94.184.143.247', c=3407),
 Row(host='40.77.167.171', c=3352),
 Row(host='94.184.143.250', c=3351),
 Row(host='82.99.202.122', c=3282),
 Row(host='92.50.41.130', c=3243),
 Row(host='157.55.39.180', c=3243),
 Row(host='157.55.39.192', c=3210),
 Row(host='157.55.39.81', c=3202),
 Row(host='5.202.255.25', c=3162),
 Row(host='207.46.13.177', c=3150),
 Row(host='92.50.40.46', c=3140),
 Row(host='40.77.167.219', c=3091),
 Row(host='78.157.44.9', c=3061),
 Row(host='5.234.161.255', c=3058),
 Row(host='185.161.113.50', c=3032),
 Row(host='77.36.152.55', c=3017),
 Row(host='46.225.182.103', c=3003),
 Row(host='80.191.103.6', c=2995),
 Row(host='185.109.80.130', c=2956),
 Row(host='178.252.143.214', c=2945),
 Row(host='185.132.214.20', c=2883),
 Row(host='5.78.128.79', c=2873),
 Row(host='134.19.177.22', c=2862),
 Row(host='2.191.96.86', c=2845),
 Row(host='194.94.127.7', c=2835),
 Row(host='81.90.144.56', c=2835),
 Row(host='81.29.241.79', c=2776),
 Row(host='5.160.1.226', c=2741),
 Row(host='207.46.13.217', c=2736),
 Row(host='178.252.129.2', c=2635),
 Row(host='46.209.45.245', c=2593),
 Row(host='2.188.27.28', c=2582),
 Row(host='134.19.177.20', c=2576),
 Row(host='134.19.177.23', c=2576),
 Row(host='94.184.141.133', c=2550),
 Row(host='65.49.68.180', c=2545),
 Row(host='207.46.13.59', c=2537),
 Row(host='157.55.39.214', c=2536),
 Row(host='207.46.13.93', c=2508),
 Row(host='188.209.79.221', c=2500),
 Row(host='37.156.241.149', c=2492),
 Row(host='207.46.13.9', c=2487),
 Row(host='85.185.36.133', c=2476),
 Row(host='157.55.39.125', c=2467),
 Row(host='94.183.36.205', c=2453),
 Row(host='91.233.57.2', c=2439),
 Row(host='46.100.209.231', c=2435),
 Row(host='50.58.192.2', c=2431),
 Row(host='46.209.149.123', c=2430),
 Row(host='134.19.177.18', c=2362),
 Row(host='5.226.55.91', c=2355),
 Row(host='46.225.129.109', c=2343),
 Row(host='2.186.181.230', c=2342),


In [0]:
#spark sql query to get the count of the logs user agent wise
sqlC.sql("select user_agent, Count(user_agent) as c from WebLogs group by user_agent order by c desc").collect()

Out[22]: [Row(user_agent='Windows', c=5137742),
 Row(user_agent='Mobile', c=3992853),
 Row(user_agent='Unknown', c=606529),
 Row(user_agent='Bot', c=606189),
 Row(user_agent='Applebot', c=21783)]

In [0]:
#spark sql query to get top viewed pages or content
sqlC.sql("select request, Count(request) as c from WebLogs group by request order by c desc limit 1").collect()

Out[23]: [Row(request='/settings/logo', c=352047)]

In [0]:
#spark sql query to know how many logs the server can not find the requested resource
sqlC.sql("select count(*) from WebLogs where rescode=404").collect()

Out[24]: [Row(count(1)=105011)]

In [0]:
#spark sql query to know count of succesful responses
sqlC.sql("select count(*) as success from WebLogs where rescode=200").collect()

Out[34]: [Row(success=9579824)]

In [0]:
#spark sql query to know the number of distinct users
sqlC.sql("select count(distinct host) from WebLogs").collect()

Out[35]: [Row(count(DISTINCT host)=258594)]

In [0]:
#spark sql query to find host whose content size greater than 20000
sqlC.sql("select host from WebLogs where size>=20000").collect()

Out[27]: [Row(host='54.36.149.41'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='178.253.33.51'),
 Row(host='91.99.72.15'),
 Row(host='207.46.13.136'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.91'),
 Row(host='178.253.33.51'),
 Row(host='5.78.198.52'),
 Row(host='178.253.33.51'),
 Row(host='207.46.13.136'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='34.247.132.53'),
 Row(host='54.36.149.70'),
 Row(host='178.253.33.51'),
 Row(host='66.249.66.194'),
 Row(host='178.253.33.51'),
 Row(host='2.177.12.140'),
 Row(host='66.249.66.91'),
 Row(host='66.111.54.249'),
 Row(host='66.249.66.194'),
 Row(host='66.111.54.249'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='91.99.72.15'),
 Row(host='17.58.102.43'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.194'),
 Row(host='91.99.72.15'),
 Row(host='51.15.15.54'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.91'),
 Row(host='31.56.96.51'),
 Row(host='54.36.149.17'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='66.111.54.249'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.17'),
 Row(host='207.46.13.104'),
 Row(host='66.111.54.249'),
 Row(host='66.249.66.91'),
 Row(host='66.111.54.249'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.32'),
 Row(host='54.36.149.35'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='157.55.39.245'),
 Row(host='91.99.72.15'),
 Row(host='207.46.13.136'),
 Row(host='91.99.72.15'),
 Row(host='157.55.39.245'),
 Row(host='66.249.66.194'),
 Row(host='91.99.72.15'),
 Row(host='54.36.149.63'),
 Row(host='5.209.200.218'),
 Row(host='31.56.96.51'),
 Row(host='207.46.13.136'),
 Row(host='5.211.97.39'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='17.58.102.43'),
 Row(host='157.55.39.245'),
 Row(host='66.249.66.194'),
 Row(host='54.36.148.10'),
 Row(host='5.209.200.218'),
 Row(host='66.249.64.66'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.104'),
 Row(host='66.111.54.249'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.117'),
 Row(host='66.249.66.91'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='54.36.148.232'),
 Row(host='91.99.72.15'),
 Row(host='91.99.72.15'),
 Row(host='204.18.198.248'),
 Row(host='54.36.149.92'),
 Row(host='204.18.198.248'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.194'),
 Row(host='51.15.15.54'),
 Row(host='66.249.66.194'),
 Row(host='17.58.102.43'),
 Row(host='5.209.200.218'),
 Row(host='54.36.149.83'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='17.58.102.43'),
 Row(host='5.78.198.52'),
 Row(host='34.247.132.53'),
 Row(host='5.78.198.52'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.197'),
 Row(host='91.99.72.15'),
 Row(host='91.99.72.15'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.104'),
 Row(host='54.36.149.84'),
 Row(host='91.99.72.15'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='31.56.96.51'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='54.36.149.105'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='66.111.54.249'),


In [0]:
#spark sql query to count host whose content size greater than 20000
sqlC.sql("select count(host) from WebLogs where size>=20000").collect()

Out[28]: [Row(count(host)=1705419)]

In [0]:
#spark sql query to find host whose content size is greater than 20000 and user_agent is bot
sqlC.sql("select host from WebLogs where user_agent='Bot' and size>=20000").collect()

Out[29]: [Row(host='54.36.149.41'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='54.36.149.70'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='54.36.149.17'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.17'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.32'),
 Row(host='54.36.149.35'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='157.55.39.245'),
 Row(host='207.46.13.136'),
 Row(host='157.55.39.245'),
 Row(host='54.36.149.63'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='157.55.39.245'),
 Row(host='54.36.148.10'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.117'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.232'),
 Row(host='54.36.149.92'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='54.36.149.83'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='54.36.149.84'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='54.36.149.105'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='54.36.149.16'),
 Row(host='66.249.66.91'),
 Row(host='54.36.149.7'),
 Row(host='66.249.66.194'),
 Row(host='54.36.149.77'),
 Row(host='54.36.149.102'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.194'),
 Row(host='54.36.148.14'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='54.36.148.197'),
 Row(host='207.46.13.136'),
 Row(host='54.36.149.54'),
 Row(host='54.36.148.70'),
 Row(host='66.249.66.91'),
 Row(host='157.55.39.245'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.104'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.135'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.57'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.176'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.194'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='66.249.66.194'),
 Row(host='207.46.13.104'),
 Row(host='157.55.39.245'),
 Row(host='66.249.66.91'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.136'),
 Row(host='207.46.13.104'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='54.36.149.86'),
 Row(host='66.249.66.91'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.143'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.118'),
 Row(host='66.249.66.194'),
 Row(host='54.36.148.65'),
 Row(host='66.249.66.91'),
 Row(host='54.36.148.22'),
 Row(host='66.249.66.91'),
 Row(host='54.36.149.68'),
 Row(host='54.36.148.23'),
 Row(host='66.249.66.91'),
 Row(host=